In [30]:
# Install all dependencies
!pip install -q requests pandas numpy plotly streamlit pyngrok cloudflared


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [31]:
# Cell 1 — imports, constants, folders
import os, json, time, pathlib, requests                       # std + HTTP + filesystem
import pandas as pd, numpy as np                               # data wrangling
from datetime import datetime                                  # timestamps for filenames

# Primary & secondary APIs
BASE_URL_CG = "https://api.coingecko.com/api/v3"               # main source
BASE_URL_PAPRIKA = "https://api.coinpaprika.com/v1"            # backup source (no key)

# Tunables
MAX_COINS = 30                                                 # how many coins to fetch/analyze
TIMEOUT   = 8                                                  # HTTP timeout (seconds)
REQUEST_DELAY = 2.0                                            # polite delay after live calls
CACHE_TTL = 600                                                # cache freshness (seconds)

# Folders
CACHE_DIR   = pathlib.Path("./cache");   CACHE_DIR.mkdir(exist_ok=True)     # on-disk json cache
OFFLINE_DIR = pathlib.Path("./offline"); OFFLINE_DIR.mkdir(exist_ok=True)   # snapshots & fallbacks

# Offline files
OFFLINE_FILE     = OFFLINE_DIR / "markets_sample.json"                       # our JSON snapshot
KAGGLE_FALLBACK  = OFFLINE_DIR / "kaggle_markets.csv"                        # optional CSV fallback

In [32]:
# Cell 2 — cache helpers + %change util
def _cache_path(key: str) -> pathlib.Path:
    return CACHE_DIR / f"{key}.json"                                        # path for a cached response

def _read_cache(key: str, ttl: int = CACHE_TTL):
    p = _cache_path(key)
    if not p.exists(): return None                                          # nothing cached
    if time.time() - p.stat().st_mtime > ttl: return None                   # stale
    try:
        return json.loads(p.read_text())                                    # parse JSON
    except Exception:
        return None                                                         # bad cache → ignore

def _write_cache(key: str, data):
    _cache_path(key).write_text(json.dumps(data))                           # save JSON

def pct_change_from_sparkline(prices, hours: int):
    """Compute % change over `hours` using the sparkline (hourly)."""
    if not prices or len(prices) <= hours: return None
    last = float(prices[-1]); prev = float(prices[-hours-1])                # ~hours points back
    if prev == 0: return None
    return 100.0 * (last - prev) / prev


In [33]:
# Cell 3 — CoinGecko (primary) + normalizer
def fetch_cg_markets(max_coins=MAX_COINS, currency="usd", ttl=CACHE_TTL):
    key = f"cg_{currency}_{max_coins}_spark"                                # cache key includes sparkline
    cached = _read_cache(key, ttl=ttl)
    if cached is not None:
        return cached, "CACHE(CG)"                                          # instant return from cache

    url = (f"{BASE_URL_CG}/coins/markets?vs_currency={currency}"
           f"&order=market_cap_desc&per_page={max_coins}&page=1"
           f"&sparkline=true&price_change_percentage=1h,24h,7d,30d")        # sparkline ON
    r = requests.get(url, timeout=TIMEOUT, headers={"Accept": "application/json"})
    if r.status_code != 200:
        raise RuntimeError(f"CG {r.status_code}")                           # bubble up to fallback
    data = r.json()
    _write_cache(key, data)                                                 # cache for next call
    time.sleep(REQUEST_DELAY)                                               # be polite to free API
    return data, "LIVE(CG)"

def norm_from_cg(row: dict) -> dict:
    """Map CoinGecko fields → our notebook schema."""
    return {
        "id": row.get("id"),
        "name": row.get("name"),
        "symbol": (row.get("symbol") or "").upper(),
        "current_price": row.get("current_price"),
        "market_cap": row.get("market_cap"),
        "volume_24h": row.get("total_volume"),                              # align naming
        "price_change_percentage_1h":  row.get("price_change_percentage_1h_in_currency", row.get("price_change_percentage_1h")),
        "price_change_percentage_24h": row.get("price_change_percentage_24h_in_currency", row.get("price_change_percentage_24h")),
        "price_change_percentage_7d":  row.get("price_change_percentage_7d_in_currency", row.get("price_change_percentage_7d")),
        "price_change_percentage_30d": row.get("price_change_percentage_30d_in_currency", row.get("price_change_percentage_30d")),
        "sparkline": (row.get("sparkline") or {}).get("price", []),         # hourly prices (~7d)
    }

# Cell 3b — CoinPaprika (secondary) + normalizer
def fetch_paprika_tickers(max_coins=MAX_COINS, ttl=CACHE_TTL):
    key = f"paprika_{max_coins}"
    cached = _read_cache(key, ttl=ttl)
    if cached is not None:
        return cached, "CACHE(PAPRIKA)"
    url = f"{BASE_URL_PAPRIKA}/tickers?quotes=USD"
    r = requests.get(url, timeout=TIMEOUT, headers={"Accept": "application/json"})
    if r.status_code != 200:
        raise RuntimeError(f"PAP {r.status_code}")
    data = r.json()
    _write_cache(key, data)
    time.sleep(REQUEST_DELAY)
    return data, "LIVE(PAPRIKA)"

def norm_from_paprika(row: dict) -> dict:
    """Map Paprika fields → our schema (no sparkline here)."""
    q = (row.get("quotes") or {}).get("USD", {})
    return {
        "id": row.get("id"),
        "name": row.get("name"),
        "symbol": (row.get("symbol") or "").upper(),
        "current_price": q.get("price"),
        "market_cap": q.get("market_cap"),
        "volume_24h": q.get("volume_24h"),
        "price_change_percentage_1h":  q.get("percent_change_1h"),
        "price_change_percentage_24h": q.get("percent_change_24h"),
        "price_change_percentage_7d":  q.get("percent_change_7d"),
        "price_change_percentage_30d": None,
        "sparkline": [],                                                    # we’ll enrich later if needed
    }

In [34]:
# Cell 4 — offline & Kaggle helpers + CDD enrichment for 48/72
def save_offline_snapshot(rows_norm: list):
    """Write normalized rows to offline JSON (best-effort)."""
    try:
        OFFLINE_FILE.write_text(json.dumps(rows_norm, indent=2))
    except Exception:
        pass

def load_offline_json() -> list:
    """Read normalized rows from offline JSON, if present."""
    if OFFLINE_FILE.exists():
        try:
            return json.loads(OFFLINE_FILE.read_text())
        except Exception:
            return []
    return []

def load_kaggle_csv() -> pd.DataFrame:
    """Optional: load a pre-normalized CSV fallback."""
    if not KAGGLE_FALLBACK.exists(): return pd.DataFrame()
    try:
        return pd.read_csv(KAGGLE_FALLBACK)
    except Exception as e:
        print("Kaggle CSV read error:", e)
        return pd.DataFrame()

# Minimal map of symbols→Binance pairs for CryptoDataDownload
CDD_MAP = {
    "BTC":"BTCUSDT","ETH":"ETHUSDT","BNB":"BNBUSDT","SOL":"SOLUSDT","XRP":"XRPUSDT",
    "ADA":"ADAUSDT","DOGE":"DOGEUSDT","TON":"TONUSDT","SHIB":"SHIBUSDT","LTC":"LTCUSDT",
    "TRX":"TRXUSDT","AVAX":"AVAXUSDT",
}

def fetch_cdd_pct_changes(symbol: str):
    """Best-effort: get 24/48/72h % change from CDD hourly CSV if API lacks sparkline."""
    pair = CDD_MAP.get(symbol.upper())
    if not pair: return None, None, None
    url = f"https://www.cryptodatadownload.com/cdd/Binance_{pair}_1h.csv"
    try:
        text = requests.get(url, timeout=TIMEOUT).text
        lines = [ln for ln in text.splitlines() if not ln.startswith("#")]         # skip comments
        if len(lines) < 10: return None, None, None
        from io import StringIO
        df = pd.read_csv(StringIO("\n".join(lines)))
        close_col = "close" if "close" in df.columns else ("Close" if "Close" in df.columns else None)
        if close_col is None: return None, None, None
        closes = df[close_col].astype(float).tolist()[::-1]                         # oldest→newest
        c24 = pct_change_from_sparkline(closes, 24)
        c48 = pct_change_from_sparkline(closes, 48)
        c72 = pct_change_from_sparkline(closes, 72)
        return c24, c48, c72
    except Exception:
        return None, None, None

def enrich_48_72_with_cdd(df: pd.DataFrame) -> pd.DataFrame:
    """Fill 48/72 (and 24 if missing) via CDD for common symbols."""
    if df.empty: return df
    df = df.copy()
    if "price_change_percentage_48h" not in df: df["price_change_percentage_48h"] = np.nan
    if "price_change_percentage_72h" not in df: df["price_change_percentage_72h"] = np.nan
    need = df["price_change_percentage_48h"].isna() | df["price_change_percentage_72h"].isna()
    for i, r in df[need].iterrows():
        c24, c48, c72 = fetch_cdd_pct_changes(r["symbol"])
        if c24 is not None and pd.isna(r.get("price_change_percentage_24h", np.nan)):
            df.at[i, "price_change_percentage_24h"] = c24
        if c48 is not None: df.at[i, "price_change_percentage_48h"] = c48
        if c72 is not None: df.at[i, "price_change_percentage_72h"] = c72
    return df

In [35]:
# Cell 5 — unified loader: CG → Paprika → Offline → Kaggle
def get_markets_multi(source="auto", max_coins=MAX_COINS, currency="usd"):
    order = ["coingecko","paprika","offline","kaggle"] if source=="auto" else [source]
    last_err = None
    for src in order:
        try:
            if src == "coingecko":
                rows, src_used = fetch_cg_markets(max_coins, currency)       # live or cache
                norm = [norm_from_cg(r) for r in rows][:max_coins]           # normalize rows
                save_offline_snapshot(norm)                                  # refresh offline
                df = pd.DataFrame(norm)

                # Compute 24/48/72 from sparkline
                df["price_change_percentage_48h"] = df["sparkline"].apply(lambda p: pct_change_from_sparkline(p, 48))
                df["price_change_percentage_72h"] = df["sparkline"].apply(lambda p: pct_change_from_sparkline(p, 72))
                df["price_change_percentage_24h"] = df.apply(
                    lambda r: r["price_change_percentage_24h"] if pd.notna(r["price_change_percentage_24h"])
                             else pct_change_from_sparkline(r["sparkline"], 24), axis=1
                )
                return df.head(max_coins), src_used

            if src == "paprika":
                rows, src_used = fetch_paprika_tickers(max_coins)            # live or cache
                norm = [norm_from_paprika(r) for r in rows][:max_coins]
                df = pd.DataFrame(norm)
                df = enrich_48_72_with_cdd(df)                               # fill 48/72 if possible
                return df.head(max_coins), src_used

            if src == "offline":
                raw = load_offline_json()                                    # load snapshot if exists
                if raw:
                    df = pd.DataFrame(raw).head(max_coins)
                    if "sparkline" in df.columns:
                        df["price_change_percentage_48h"] = df["sparkline"].apply(lambda p: pct_change_from_sparkline(p, 48))
                        df["price_change_percentage_72h"] = df["sparkline"].apply(lambda p: pct_change_from_sparkline(p, 72))
                    return df, "OFFLINE(JSON)"

            if src == "kaggle":
                kag = load_kaggle_csv()                                      # final fallback
                if not kag.empty:
                    return kag.head(max_coins), "OFFLINE(KAGGLE)"

        except Exception as e:
            last_err = e                                                     # remember and try next
            continue

    raise RuntimeError(f"All sources failed: {last_err}")                    # bubble up

In [36]:
# Cell 6 — feature helpers (same names; safe to reuse elsewhere)

def calculate_volatility_proxy(coin_row: dict):
    """Mean absolute % moves across 1h, 24h, 7d (simple volatility proxy)."""
    vals = []
    for k in ["price_change_percentage_1h","price_change_percentage_24h","price_change_percentage_7d"]:
        v = coin_row.get(k)
        if v is not None: vals.append(abs(v))
    return None if not vals else float(np.mean(vals))

def simple_trend_forecast(coin_row: dict):
    """Toy forecast: nudge current price by 30% of (0.7*24h + 0.3*7d). Demo only."""
    c24 = coin_row.get("price_change_percentage_24h", 0) or 0
    c7  = coin_row.get("price_change_percentage_7d", 0)  or 0
    cur = coin_row.get("current_price", 0) or 0
    if not cur: return None
    trend = 0.7*c24 + 0.3*c7
    return round(cur * (1 + 0.3*(trend/100)), 6)

def compute_risk_score(row: dict):
    """Absolute rule score out of 100 + reason (independent from normalized risk)."""
    score, reasons = 0, []
    if row.get("price_change_percentage_24h") is not None and row["price_change_percentage_24h"] < -2:
        score += 50; reasons.append("24h drop > 2%")
    if row.get("volatility_proxy") is not None and row["volatility_proxy"] > 3:
        score += 30; reasons.append("High recent volatility")
    if row.get("volume_24h") and row.get("market_cap") and row["volume_24h"] > 0.5*row["market_cap"]:
        score += 15; reasons.append("Heavy 24h volume vs market cap")
    if row.get("price_change_percentage_1h") is not None and row["price_change_percentage_1h"] < -0.5:
        score += 20; reasons.append("1h downside")
    if score == 0: reasons.append("Stable / normal range")
    return score, "; ".join(reasons)

In [37]:
# Cell 7 — main analysis pipeline
def analyze_free_data(source="auto"):
    """
    1) Load markets via multi-source fallback
    2) Assemble per-coin dicts with raw + derived features
    3) Clean dtypes
    4) Compute normalized cohort risk (0–100)
    5) Human-readable reasons
    """
    # 1) fetch with fallbacks
    market_df, source_used = get_markets_multi(source=source, max_coins=MAX_COINS, currency="usd")
    print(f"[DATA SOURCE] {source_used} ✓")
    if market_df is None or market_df.empty:
        print("No market data available.")
        return pd.DataFrame()

    # Convert to records so we can reuse existing row logic easily
    records = market_df.to_dict("records")

    # 2) per-coin features
    results = []
    for coin in records:
        # core fields (safe gets)
        coin_id = coin.get("id")
        name    = coin.get("name")
        symbol  = (coin.get("symbol") or "").upper()
        price   = coin.get("current_price")
        mcap    = coin.get("market_cap", 0)
        vol24   = coin.get("volume_24h", 0)

        # % changes (API + sparkline/CDD backfills already done in loader)
        c1   = coin.get("price_change_percentage_1h")
        c24  = coin.get("price_change_percentage_24h")
        c48  = coin.get("price_change_percentage_48h")
        c72  = coin.get("price_change_percentage_72h")
        c7d  = coin.get("price_change_percentage_7d")
        c30d = coin.get("price_change_percentage_30d")

        # volatility proxy + toy forecast
        vol_proxy = calculate_volatility_proxy(coin)
        forecast  = simple_trend_forecast(coin)

        # collect row
        results.append({
            "coin_id": coin_id, "name": name, "symbol": symbol,
            "current_price": price, "market_cap": mcap, "volume_24h": vol24,
            "price_change_percentage_1h": c1,
            "price_change_percentage_24h": c24,
            "price_change_percentage_48h": c48,
            "price_change_percentage_72h": c72,
            "price_change_percentage_7d": c7d,
            "price_change_percentage_30d": c30d,
            "volatility_proxy": vol_proxy, "trend_forecast": forecast,
        })

    # 3) DataFrame + type coercion
    df = pd.DataFrame(results)
    for col in [
        "current_price","market_cap","volume_24h",
        "price_change_percentage_1h","price_change_percentage_24h",
        "price_change_percentage_48h","price_change_percentage_72h",
        "price_change_percentage_7d","price_change_percentage_30d",
        "volatility_proxy","trend_forecast"
    ]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # 4) normalized risk components (sigmoid of z-scores)
    df["abs_24h"]     = df["price_change_percentage_24h"].abs()
    df["down_1h"]     = df["price_change_percentage_1h"].clip(upper=0).abs()
    df["volume_ratio"]= (df["volume_24h"]/df["market_cap"]).clip(lower=0, upper=1)

    def _norm(s: pd.Series) -> pd.Series:
        s2 = s.replace([np.inf,-np.inf], np.nan)
        m, sd = s2.mean(), s2.std()
        if not np.isfinite(sd) or sd == 0: return pd.Series(0, index=s.index)
        z = (s2 - m) / sd
        return 1/(1+np.exp(-z))

    risk = (0.50*_norm(df["volatility_proxy"]) +
            0.30*_norm(df["abs_24h"]) +
            0.15*_norm(df["down_1h"]) +
            0.05*_norm(df["volume_ratio"]))
    df["risk_score"] = (100*risk).clip(0,100).round(1)

    # 5) reasons
    reasons = []
    for _, r in df.iterrows():
        why=[]
        if r["volatility_proxy"] > df["volatility_proxy"].median(): why.append("High recent volatility")
        if r["abs_24h"]          > df["abs_24h"].median():         why.append("Large 24h move")
        if r["price_change_percentage_1h"] < -0.5:                  why.append("1h downside")
        if r["volume_ratio"] > 0.5:                                 why.append("Heavy volume vs mcap")
        reasons.append("; ".join(why) if why else "Stable / normal range")
    df["risk_reason"] = reasons

    return df

In [38]:
# Cell 8 — pretty console table
def display_results(df: pd.DataFrame):
    if df is None or df.empty:
        print("No data to display."); return

    df_sorted = df.sort_values("risk_score", ascending=False)
    header = f"{'Name':<20} {'Sym':<6} {'Price($)':>10} {'24h%':>8} {'48h%':>8} {'72h%':>8} {'VolProxy':>10} {'Risk%':>7}  Reason"
    print(header)
    print('-'*len(header))
    for _, r in df_sorted.iterrows():
        print(f"{str(r['name'])[:20]:<20} {r['symbol']:<6} "
              f"{(r['current_price'] or 0):>10.4f} "
              f"{(r.get('price_change_percentage_24h') or 0):>8.2f} "
              f"{(r.get('price_change_percentage_48h') or 0):>8.2f} "
              f"{(r.get('price_change_percentage_72h') or 0):>8.2f} "
              f"{(r['volatility_proxy'] or 0):>10.2f} "
              f"{r['risk_score']:>7.1f}  "
              f"{r['risk_reason']}")


In [39]:
# Cell 9 — Plotly charts (save to HTML; show inline if you want)
import plotly.express as px
import plotly.graph_objects as go

def create_visualizations(df: pd.DataFrame):
    if df is None or df.empty:
        print("No data for charts."); return

    # 1) Top-10 volatility
    top_vol = df.nlargest(10, "volatility_proxy")
    fig1 = px.bar(top_vol, x="symbol", y="volatility_proxy", title="Top-10 Volatility Proxy")
    fig1.write_html("volatility_proxy_chart.html")

    # 2) 24h vs 7d grouped
    fig2 = go.Figure()
    fig2.add_bar(name="24h %", x=df["symbol"], y=df["price_change_percentage_24h"])
    fig2.add_bar(name="7d %",  x=df["symbol"], y=df["price_change_percentage_7d"])
    fig2.update_layout(barmode="group", title="Price Change: 24h vs 7d")
    fig2.write_html("price_changes_chart.html")

    # 3) 24/48/72 grouped (new)
    figX = go.Figure()
    figX.add_bar(name="24h %", x=df["symbol"], y=df["price_change_percentage_24h"])
    figX.add_bar(name="48h %", x=df["symbol"], y=df["price_change_percentage_48h"])
    figX.add_bar(name="72h %", x=df["symbol"], y=df["price_change_percentage_72h"])
    figX.update_layout(barmode="group", title="Price Change: 24/48/72h")
    figX.write_html("price_changes_24_48_72.html")

    # 4) Market cap vs 24h volume (log-log)
    fig3 = px.scatter(df, x="market_cap", y="volume_24h", log_x=True, log_y=True,
                      hover_name="name", title="Market Cap vs 24h Volume (log-log)")
    fig3.write_html("market_analysis_chart.html")

    # 5) Top-10 risk
    top_risk = df.nlargest(10, "risk_score")
    fig4 = px.bar(top_risk, x="symbol", y="risk_score", title="Top-10 Crash Risk")
    fig4.write_html("risk_score_chart.html")

    print("Saved charts: volatility_proxy_chart.html, price_changes_chart.html, price_changes_24_48_72.html, market_analysis_chart.html, risk_score_chart.html")


In [40]:
# Cell 10 — CSV export
def save_results(df: pd.DataFrame):
    if df is None or df.empty:
        return
    df_export = df.copy().round(6)
    filename = f"crypto_analysis_{datetime.now():%Y%m%d_%H%M}.csv"
    df_export.to_csv(filename, index=False)
    print(f"Results saved → {filename}")


In [41]:
# Cell 11 — main orchestration with logs
def main(source="auto"):
    print("=== Crashcaster pipeline start ===")
    try:
        print("[1/4] Fetch + analyze …")
        df = analyze_free_data(source=source)
        if df is None or df.empty:
            print("!! Empty DataFrame."); return df

        print("[2/4] Print table …")
        try: display_results(df)
        except Exception as e: print("display_results error:", e)

        print("[3/4] Charts …")
        try: create_visualizations(df)
        except Exception as e: print("create_visualizations error:", e)

        print("[4/4] Save CSV …")
        try: save_results(df)
        except Exception as e: print("save_results error:", e)

        print("=== Done ===")
        return df
    except Exception as e:
        import traceback; traceback.print_exc()
        return None

# Run once to populate `results_df`
results_df = main(source="auto")

=== Crashcaster pipeline start ===
[1/4] Fetch + analyze …
[DATA SOURCE] LIVE(CG) ✓
[2/4] Print table …
Name                 Sym      Price($)     24h%     48h%     72h%   VolProxy   Risk%  Reason
--------------------------------------------------------------------------------------------
Chainlink            LINK      22.4600    -6.46      nan      nan       9.71    74.6  High recent volatility; Large 24h move
Solana               SOL      193.6900    -5.74      nan      nan       5.67    69.3  High recent volatility; Large 24h move
Cardano              ADA        0.9248    -4.71      nan      nan       7.30    65.1  High recent volatility; Large 24h move
Dogecoin             DOGE       0.2268    -9.58      nan      nan       4.17    61.3  High recent volatility; Large 24h move
Ethereum             ETH     4633.4600    -2.54      nan      nan       7.31    59.3  High recent volatility
Stellar              XLM        0.4276    -7.85      nan      nan       4.38    59.1  High recent vol

In [42]:
# Cell 12 — risk tools + horizon mapping
risk_level = 60                                           # demo default
time_horizon = "24h"                                      # "24h" | "48h" | "72h" | "7d"

def risk_to_volatility_threshold(risk_level: int) -> float:
    risk_level = max(10, min(100, risk_level))            # clamp
    return 1 + (risk_level - 10) * (9 / 90.0)             # linear 1..10

def get_change_column(time_horizon: str) -> str:
    mapping = {
        "24h": "price_change_percentage_24h",
        "48h": "price_change_percentage_48h",
        "72h": "price_change_percentage_72h",
        "7d":  "price_change_percentage_7d",
    }
    return mapping.get(time_horizon, "price_change_percentage_24h")

def show_risk_based_recommendations(df: pd.DataFrame, risk_level: int, time_horizon: str):
    """Filter coins by volatility tolerance and show chosen horizon column."""
    if df is None or df.empty:
        print("No data for recommendations."); return
    vol_thresh = risk_to_volatility_threshold(risk_level)
    col = get_change_column(time_horizon)
    dff = df[df["volatility_proxy"] <= vol_thresh].copy()
    if dff.empty:
        print("No coins within your risk tolerance."); return
    header = f"{'Name':<20} {'Sym':<6} {'VolProxy':>9} {col:>12}  Reason"
    print("\nRecommended (within your volatility tolerance):")
    print(header); print("-"*len(header))
    for _, r in dff.sort_values("volatility_proxy").iterrows():
        print(f"{str(r['name'])[:20]:<20} {r['symbol']:<6} "
              f"{(r['volatility_proxy'] or 0):>9.2f} "
              f"{(r.get(col) or 0):>12.2f}  "
              f"{r['risk_reason']}")

# Example:
show_risk_based_recommendations(results_df, risk_level=risk_level, time_horizon="24h")


Recommended (within your volatility tolerance):
Name                 Sym     VolProxy price_change_percentage_24h  Reason
-------------------------------------------------------------------------
USDC                 USDC        0.00        -0.00  Stable / normal range
USDS                 USDS        0.03         0.03  Stable / normal range
Tether               USDT        0.05         0.06  Heavy volume vs mcap
Ethena USDe          USDE        0.07         0.13  Stable / normal range
Binance Bridged USDT BSC-USD      0.11         0.09  Stable / normal range
Wrapped Bitcoin      WBTC        1.73        -3.71  Large 24h move
Bitcoin              BTC         1.88        -3.96  Large 24h move
Toncoin              TON         2.29        -4.05  Large 24h move
LEO Token            LEO         2.45         1.67  Stable / normal range
Bitcoin Cash         BCH         2.58        -4.47  Large 24h move
BNB                  BNB         2.67        -0.86  Stable / normal range
TRON             

In [43]:
# Cell 13 — rules-based buy recommendations (explainable demo)
def recommend_coins(df: pd.DataFrame, strategy="trend", top_k=10):
    d = df.copy()
    # ensure presence of needed columns
    for col in ["price_change_percentage_24h","price_change_percentage_48h",
                "price_change_percentage_72h","volatility_proxy","risk_score"]:
        if col not in d.columns: d[col] = np.nan

    if strategy == "trend":
        mask = (
            (d["risk_score"] < 65) &
            (d["volatility_proxy"] < d["volatility_proxy"].quantile(0.7)) &
            (d["price_change_percentage_24h"] > 0) &
            (d["price_change_percentage_48h"] > 0) &
            (d["price_change_percentage_72h"] >= 0)
        )
        score = (0.5*d["price_change_percentage_24h"] +
                 0.3*d["price_change_percentage_48h"] +
                 0.2*d["price_change_percentage_72h"] -
                 0.2*d["volatility_proxy"])

    elif strategy == "reversal":
        mask = (
            (d["risk_score"] < 55) &
            (d["price_change_percentage_24h"] < 0) &
            (d["price_change_percentage_72h"] > 0) &
            (d["volatility_proxy"] < d["volatility_proxy"].quantile(0.8))
        )
        score = ((-1.0)*d["price_change_percentage_24h"] +
                 0.3*d["price_change_percentage_72h"] -
                 0.2*d["volatility_proxy"])
    else:
        raise ValueError("strategy must be 'trend' or 'reversal'")

    out = d[mask].copy()
    out["recommend_score"] = score[mask]
    out = out.sort_values("recommend_score", ascending=False)
    return out.head(top_k)[[
        "name","symbol","current_price",
        "price_change_percentage_24h","price_change_percentage_48h","price_change_percentage_72h",
        "volatility_proxy","risk_score","risk_reason","recommend_score"
    ]]

def show_recommendations(df: pd.DataFrame):
    print("\n=== BUY (Trend) ===")
    print(recommend_coins(df, "trend", top_k=10).to_string(index=False))
    print("\n=== BUY (Reversal) ===")
    print(recommend_coins(df, "reversal", top_k=10).to_string(index=False))

# Example:
show_recommendations(results_df)


=== BUY (Trend) ===
Empty DataFrame
Columns: [name, symbol, current_price, price_change_percentage_24h, price_change_percentage_48h, price_change_percentage_72h, volatility_proxy, risk_score, risk_reason, recommend_score]
Index: []

=== BUY (Reversal) ===
Empty DataFrame
Columns: [name, symbol, current_price, price_change_percentage_24h, price_change_percentage_48h, price_change_percentage_72h, volatility_proxy, risk_score, risk_reason, recommend_score]
Index: []


In [44]:
# Cell A — write polished Streamlit app
with open("app.py", "w") as f:
    f.write('''import os, json, time, pathlib, requests
import numpy as np
import pandas as pd
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go

# -------- App config --------
st.set_page_config(page_title="Crashcaster — Early Warning", page_icon="💥", layout="wide")

# --- UX polish (hide chrome, badges) ---
HIDE_DEFAULT = """
<style>
#MainMenu {visibility: hidden;}
footer {visibility: hidden;}
header {visibility: hidden;}
section[data-testid="stSidebar"] {background:#0E1117;}
div.block-container {padding-top: 1.0rem;}
.badge {display:inline-block;padding:6px 10px;border-radius:999px;font-weight:600;margin-right:6px;}
.badge.green{background:#1f6f43;color:#fff;}
.badge.orange{background:#a86b16;color:#fff;}
.badge.red{background:#a11f2c;color:#fff;}
</style>
"""
st.markdown(HIDE_DEFAULT, unsafe_allow_html=True)

def risk_badge_class(x):
    return "red" if x>=70 else ("orange" if x>=40 else "green")

def reason_badges(reason_text: str):
    parts = [p.strip() for p in (reason_text or "").split(";") if p.strip()]
    html = "".join([f"<span class='badge {'red' if ('High' in p or 'Large' in p) else 'green'}'>{p}</span>" for p in parts])
    return html or "<span class='badge green'>Stable</span>"

# -------- Data config --------
BASE_URL_CG = "https://api.coingecko.com/api/v3"
CACHE_DIR   = pathlib.Path("./cache");   CACHE_DIR.mkdir(exist_ok=True)
OFFLINE_DIR = pathlib.Path("./offline"); OFFLINE_DIR.mkdir(exist_ok=True)
OFFLINE_FILE = OFFLINE_DIR / "markets_sample.json"
TIMEOUT, REQUEST_DELAY, CACHE_TTL = 8, 2.0, 600

# -------- Small utils --------
def _cache_path(key): return CACHE_DIR / f"{key}.json"
def _read_cache(key, ttl=CACHE_TTL):
    p = _cache_path(key)
    if not p.exists(): return None
    if time.time() - p.stat().st_mtime > ttl: return None
    try: return json.loads(p.read_text())
    except: return None
def _write_cache(key, data): _cache_path(key).write_text(json.dumps(data))

def pct_change_from_sparkline(prices, hours):
    if not prices or len(prices) <= hours: return None
    last = float(prices[-1]); prev = float(prices[-hours-1])
    if prev == 0: return None
    return 100.0 * (last - prev) / prev

# -------- Primary source (CoinGecko) --------
def fetch_cg_markets(max_coins=30, currency="usd", ttl=CACHE_TTL):
    key=f"cg_{currency}_{max_coins}_spark"
    cached = _read_cache(key, ttl=ttl)
    if cached is not None:
        return cached, "CACHE(CG)"
    url=(f"{BASE_URL_CG}/coins/markets?vs_currency={currency}"
         f"&order=market_cap_desc&per_page={max_coins}&page=1"
         f"&sparkline=true&price_change_percentage=1h,24h,7d,30d")
    r = requests.get(url, timeout=TIMEOUT, headers={"Accept":"application/json"})
    if r.status_code != 200:
        raise RuntimeError(f"CG {r.status_code}")
    data = r.json()
    _write_cache(key, data)
    time.sleep(REQUEST_DELAY)
    return data, "LIVE(CG)"

def norm_from_cg(row):
    return {
        "id": row.get("id"),
        "name": row.get("name"),
        "symbol": (row.get("symbol") or "").upper(),
        "current_price": row.get("current_price"),
        "market_cap": row.get("market_cap"),
        "volume_24h": row.get("total_volume"),
        "price_change_percentage_1h":  row.get("price_change_percentage_1h_in_currency", row.get("price_change_percentage_1h")),
        "price_change_percentage_24h": row.get("price_change_percentage_24h_in_currency", row.get("price_change_percentage_24h")),
        "price_change_percentage_7d":  row.get("price_change_percentage_7d_in_currency", row.get("price_change_percentage_7d")),
        "price_change_percentage_30d": row.get("price_change_percentage_30d_in_currency", row.get("price_change_percentage_30d")),
        "sparkline": (row.get("sparkline") or {}).get("price", []),
    }

def load_offline_json():
    if OFFLINE_FILE.exists():
        try: return json.loads(OFFLINE_FILE.read_text())
        except: return []
    return []

def get_markets_with_fallback(source="auto", max_coins=30, currency="usd"):
    last_err = None
    if source in ("auto","primary"):
        try:
            rows, src = fetch_cg_markets(max_coins, currency)
            norm = [norm_from_cg(r) for r in rows][:max_coins]
            try: OFFLINE_FILE.write_text(json.dumps(norm, indent=2))
            except: pass
            df = pd.DataFrame(norm)
            df["price_change_percentage_48h"] = df["sparkline"].apply(lambda p: pct_change_from_sparkline(p,48))
            df["price_change_percentage_72h"] = df["sparkline"].apply(lambda p: pct_change_from_sparkline(p,72))
            df["price_change_percentage_24h"] = df.apply(
                lambda r: r["price_change_percentage_24h"] if pd.notna(r["price_change_percentage_24h"])
                else pct_change_from_sparkline(r["sparkline"],24), axis=1)
            return df, src, None
        except Exception as e:
            last_err = e
    raw = load_offline_json()
    if raw:
        df = pd.DataFrame(raw).head(max_coins)
        if "sparkline" in df.columns:
            df["price_change_percentage_48h"] = df["sparkline"].apply(lambda p: pct_change_from_sparkline(p,48))
            df["price_change_percentage_72h"] = df["sparkline"].apply(lambda p: pct_change_from_sparkline(p,72))
        return df, "OFFLINE(JSON)", last_err
    raise RuntimeError(f"No data sources available. Last live error: {last_err}")

# -------- Features + risk --------
def _coerce_numeric(df):
    for c in ["current_price","market_cap","volume_24h",
              "price_change_percentage_1h","price_change_percentage_24h",
              "price_change_percentage_48h","price_change_percentage_72h",
              "price_change_percentage_7d","price_change_percentage_30d"]:
        if c in df.columns: df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def _norm(s: pd.Series) -> pd.Series:
    s2 = s.replace([np.inf,-np.inf], np.nan)
    m, sd = s2.mean(), s2.std()
    if not np.isfinite(sd) or sd == 0: return pd.Series(0, index=s.index)
    z = (s2 - m) / sd
    return 1/(1+np.exp(-z))

def build_features_and_risk(df: pd.DataFrame) -> pd.DataFrame:
    df = _coerce_numeric(df.copy())
    df["abs_24h"]      = df["price_change_percentage_24h"].abs()
    df["down_1h"]      = df["price_change_percentage_1h"].clip(upper=0).abs()
    df["volume_ratio"] = (df["volume_24h"]/df["market_cap"]).clip(lower=0, upper=1)
    df["volatility_proxy"] = (
        df["price_change_percentage_1h"].abs().fillna(0) +
        df["price_change_percentage_24h"].abs().fillna(0) +
        df["price_change_percentage_7d"].abs().fillna(0)
    )/3.0
    risk = (0.50*_norm(df["volatility_proxy"]) +
            0.30*_norm(df["abs_24h"]) +
            0.15*_norm(df["down_1h"]) +
            0.05*_norm(df["volume_ratio"]))
    df["risk_score"] = (100*risk).clip(0,100).round(1)
    # reasons
    reasons=[]
    for _, r in df.iterrows():
        why=[]
        if r["volatility_proxy"] > df["volatility_proxy"].median(): why.append("High recent volatility")
        if r["abs_24h"] > df["abs_24h"].median(): why.append("Large 24h move")
        if r["price_change_percentage_1h"] < -0.5: why.append("1h downside")
        if r["volume_ratio"] > 0.5: why.append("Heavy volume vs mcap")
        reasons.append("; ".join(why) if why else "Stable / normal range")
    df["risk_reason"] = reasons
    return df

# -------- Sidebar --------
with st.sidebar:
    st.header("Settings")
    source = st.selectbox("Data source", ["auto","primary","offline"], index=0)
    max_coins = st.slider("Coins", 10, 100, 30)
    threshold = st.slider("Risk threshold", 0, 100, 70)

    st.caption("Utilities")
    if st.button("Refresh offline snapshot"):
        try:
            rows, _ = fetch_cg_markets(max_coins=max_coins, currency="usd")
            norm = [norm_from_cg(r) for r in rows][:max_coins]
            OFFLINE_FILE.write_text(json.dumps(norm, indent=2))
            st.success("Offline snapshot refreshed ✓")
        except Exception as e:
            st.error(f"Live fetch failed: {e}")
    if st.button("Clear API cache"):
        import pathlib
        for p in pathlib.Path("cache").glob("*.json"): p.unlink()
        st.success("Cache cleared.")

@st.cache_data(ttl=600, show_spinner=False)
def load_and_score(src, n):
    df, src_used, last_err = get_markets_with_fallback(source=src, max_coins=n)
    df = build_features_and_risk(df).sort_values("risk_score", ascending=False).reset_index(drop=True)
    return df, src_used, last_err

# -------- Main --------
st.title("💥 Crashcaster — Early Warning for Crypto Crashes")
df, src_used, last_err = load_and_score(source, max_coins)

# KPIs
col1, col2, col3, col4 = st.columns(4)
if not df.empty:
    top = df.iloc[0]
    col1.metric("Top Risk Coin", f"{top['symbol']}", f"{top['risk_score']:.0f}")
    col2.metric("Market Risk Avg", f"{df['risk_score'].mean():.0f}%")
    col3.metric("Coins ≥ Threshold", f"{(df['risk_score']>=threshold).sum()}/{len(df)}")
    col4.metric("Data Source", src_used)

if src_used.startswith("OFFLINE") and last_err:
    st.warning(f"Using offline snapshot. Live fetch failed with: {last_err}")

# -------- Hero “Try it now” panel --------
st.markdown("### Try it now")
left, right = st.columns([3,1])
with left:
    symbols = df["symbol"].dropna().unique().tolist()
    symbols.sort()
    selected_symbol = st.selectbox("Pick a coin", symbols, index=0)
with right:
    analyze_clicked = st.button("Analyze", use_container_width=True)

if analyze_clicked:
    st.session_state["selected_symbol"] = selected_symbol

if "selected_symbol" in st.session_state:
    sel = st.session_state["selected_symbol"]
    row = df[df["symbol"] == sel].head(1)
    if not row.empty:
        r = row.iloc[0]
        c1, c2 = st.columns([2,1])
        with c1:
            st.subheader(f"{r['name']} ({r['symbol']})")
            spark = r.get("sparkline", None)
            if isinstance(spark, (list, tuple)) and len(spark) > 5:
                fig = go.Figure(go.Scatter(y=spark, mode="lines"))
                fig.update_layout(height=200, margin=dict(l=10,r=10,t=10,b=10), showlegend=False)
                st.plotly_chart(fig, use_container_width=True)
            s24 = r.get("price_change_percentage_24h", 0) or 0
            s48 = r.get("price_change_percentage_48h", np.nan)
            s72 = r.get("price_change_percentage_72h", np.nan)
            st.markdown(
                f"**24h:** {s24:+.2f}% · **48h:** {s48 if pd.notna(s48) else '—'}% · "
                f"**72h:** {s72 if pd.notna(s72) else '—'}%"
            )
        with c2:
            gauge = go.Figure(go.Indicator(mode="gauge+number", value=float(r["risk_score"]),
                                           gauge={"axis":{"range":[0,100]}}, title={"text":"Risk score"}))
            gauge.update_layout(height=220, margin=dict(l=10,r=10,t=30,b=10))
            st.plotly_chart(gauge, use_container_width=True)
            st.markdown(f"<div>{reason_badges(r['risk_reason'])}</div>", unsafe_allow_html=True)

# -------- Tabs --------
tab1, tab2 = st.tabs(["📊 Dashboard","✅ Recommendations"])

with tab1:
    st.subheader("At-Risk Coins")
    danger = df[df["risk_score"]>=threshold]
    st.dataframe(danger[[
        "name","symbol","current_price",
        "price_change_percentage_24h","price_change_percentage_48h","price_change_percentage_72h",
        "price_change_percentage_7d",
        "volatility_proxy","volume_24h","market_cap","risk_score","risk_reason"
    ]], use_container_width=True)

    st.subheader("All Coins")
    st.dataframe(df[[
        "name","symbol","current_price",
        "price_change_percentage_24h","price_change_percentage_48h","price_change_percentage_72h",
        "price_change_percentage_7d",
        "volatility_proxy","volume_24h","market_cap","risk_score","risk_reason"
    ]], use_container_width=True)

    st.subheader("Charts")
    top_risk = df.nlargest(10, "risk_score")
    fig1 = px.bar(top_risk, x="symbol", y="risk_score", title="Top-10 Crash Risk")
    st.plotly_chart(fig1, use_container_width=True)
    fig2 = go.Figure()
    fig2.add_bar(name="24h %", x=df["symbol"], y=df["price_change_percentage_24h"])
    fig2.add_bar(name="48h %", x=df["symbol"], y=df["price_change_percentage_48h"])
    fig2.add_bar(name="72h %", x=df["symbol"], y=df["price_change_percentage_72h"])
    fig2.update_layout(barmode="group", title="Price Change: 24/48/72h")
    st.plotly_chart(fig2, use_container_width=True)

with tab2:
    def recommend_coins_local(dff, strategy="trend", top_k=10):
        d = dff.copy()
        for col in ["price_change_percentage_24h","price_change_percentage_48h",
                    "price_change_percentage_72h","volatility_proxy","risk_score"]:
            if col not in d.columns: d[col] = np.nan
            d[col] = pd.to_numeric(d[col], errors="coerce")
        c24 = d["price_change_percentage_24h"].fillna(0)
        c48 = d["price_change_percentage_48h"].fillna(0)
        c72 = d["price_change_percentage_72h"].fillna(0)
        vol = d["volatility_proxy"].fillna(d["volatility_proxy"].median())
        def ok_nonneg_or_nan(s): return (s >= 0) | (s.isna())
        if strategy == "trend":
            mask = ((d["risk_score"] < 65) & (vol < vol.quantile(0.7)) &
                    (c24 > 0) & ok_nonneg_or_nan(d["price_change_percentage_48h"]) &
                    ok_nonneg_or_nan(d["price_change_percentage_72h"]))
            score = (0.60*c24 + 0.25*c48 + 0.15*c72 - 0.20*vol)
        else:  # reversal
            mask = ((d["risk_score"] < 55) & (c24 < 0) &
                    ok_nonneg_or_nan(d["price_change_percentage_72h"]) &
                    (vol < vol.quantile(0.8)))
            score = ((-1.0)*c24 + 0.30*c72 - 0.20*vol)
        out = d[mask].copy()
        out["recommend_score"] = score[mask]
        out = out.sort_values("recommend_score", ascending=False).head(top_k)
        if out.empty:
            fb = d[(d["risk_score"] < 60)].copy()
            fb["recommend_score"] = 0.70*c24 - 0.20*vol
            out = fb.sort_values("recommend_score", ascending=False).head(top_k)
            st.info("Showing fallback list (limited 48/72h data). Refresh the offline snapshot when online to improve results.")
        cols = ["name","symbol","current_price",
                "price_change_percentage_24h","price_change_percentage_48h","price_change_percentage_72h",
                "volatility_proxy","risk_score","risk_reason","recommend_score"]
        return out[cols]

    strat = st.selectbox("Strategy", ["trend","reversal"], index=0)
    rec = recommend_coins_local(df, strat, top_k=10)
    st.dataframe(rec, use_container_width=True)

st.caption("Tip: If the API hiccups, create an offline snapshot (sidebar) and set Data source → 'offline'. Not financial advice.")
''')


In [48]:
# ✅ Start Streamlit (subprocess), wait for port 8501, then open ngrok tunnel

import sys, subprocess, shlex, time, socket, pathlib
from contextlib import closing

PORT = 8501
NGROK_AUTHTOKEN = "31IxW8d8joEMdtLFoWy59u5f8Fb_V1gsdmuc6T4RY13aHW8P"  # rotate later

# 0) Ensure deps
try:
    import streamlit  # noqa
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "streamlit"])
try:
    from pyngrok import ngrok, conf
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "pyngrok"])
    from pyngrok import ngrok, conf

# 1) Kill any old ngrok agent/tunnels (avoid “too many tunnels” / stale sessions)
try:
    for t in ngrok.get_tunnels():
        ngrok.disconnect(t.public_url)
    ngrok.kill()
except Exception:
    pass

# 2) Launch Streamlit as a subprocess (bind to 0.0.0.0 so ngrok can reach it)
pathlib.Path("logs").mkdir(exist_ok=True)
log_f = open("logs/streamlit.log", "w")
cmd = f'{sys.executable} -m streamlit run app.py --server.port {PORT} --server.address 0.0.0.0 --server.headless true'
proc = subprocess.Popen(shlex.split(cmd), stdout=log_f, stderr=subprocess.STDOUT)

# Store handles so you can stop later
globals()["_STREAMLIT_PROC"] = proc
globals()["_STREAMLIT_LOG"] = log_f

# 3) Wait until the port is actually accepting connections
def wait_for_port(port, host="127.0.0.1", timeout=60):
    t0 = time.time()
    while time.time() - t0 < timeout:
        with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as sock:
            sock.settimeout(1.0)
            if sock.connect_ex((host, port)) == 0:
                return True
        time.sleep(0.5)
    return False

ok = wait_for_port(PORT)
if not ok:
    print("Streamlit failed to start on port", PORT)
    print("Check logs with: !tail -n 200 logs/streamlit.log")
else:
    # 4) Open ngrok tunnel only after Streamlit is up
    conf.get_default().auth_token = NGROK_AUTHTOKEN
    tunnel = ngrok.connect(PORT)
    globals()["_NGROK_TUNNEL"] = tunnel
    print("Public URL:", tunnel.public_url)
    print("Streamlit PID:", proc.pid)
    print("Tail logs with:  !tail -n 200 logs/streamlit.log")


Public URL: https://9aeb9ffdd2e4.ngrok-free.app
Streamlit PID: 14057
Tail logs with:  !tail -n 200 logs/streamlit.log


t=2025-08-14T22:59:13-0400 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"
